# SpeedController Analysis

This notebook analyzes the behavior of the SpeedController class by visualizing the wheel speeds as 3D surfaces based on joystick input coordinates.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import subprocess
import os

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Generate test data by running the Node.js script
script_path = 'test-speed-controller.js'
csv_path = 'speed_controller_test.csv'

print("Running SpeedController test...")
result = subprocess.run(['node', script_path], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

Running SpeedController test...
Testing SpeedController...
Generated 441 test points
CSV output saved to: /Users/eric/proj/microbit/pxt-cutebot/tests/speed_controller_test.csv
Sample data:
[
  {
    x: 0,
    y: 0,
    lw_speed: -66.66666666666666,
    rw_speed: -133.33333333333334,
    forward_speed: -100,
    turn_speed: 33.333333333333336
  },
  {
    x: 0,
    y: 51,
    lw_speed: -50.04887585532747,
    rw_speed: -130.00977517106548,
    forward_speed: -90.02932551319648,
    turn_speed: 39.98044965786901
  },
  {
    x: 0,
    y: 102,
    lw_speed: -33.43108504398828,
    rw_speed: -126.68621700879766,
    forward_speed: -80.05865102639297,
    turn_speed: 46.62756598240468
  },
  {
    x: 0,
    y: 153,
    lw_speed: -16.813294232649078,
    rw_speed: -123.36265884652983,
    forward_speed: -70.08797653958945,
    turn_speed: 53.27468230694037
  },
  {
    x: 0,
    y: 204,
    lw_speed: -0.19550342130987985,
    rw_speed: -120.03910068426197,
    forward_speed: -60.117302052785

In [3]:
# Load the CSV data
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} data points")
print("\nData summary:")
print(df.describe())
print("\nFirst few rows:")
print(df.head())

Loaded 441 data points

Data summary:
                 x            y    lw_speed    rw_speed  forward_speed  \
count   441.000000   441.000000  441.000000  441.000000     441.000000   
mean    510.000000   510.000000   -0.102134   -0.484377      -0.293255   
std     309.171069   309.171069   69.859294   69.858435      60.444002   
min       0.000000     0.000000 -132.942326 -133.333333    -100.000000   
25%     255.000000   255.000000  -61.915317  -62.237382     -50.146628   
50%     510.000000   510.000000   -0.168772   -0.564796      -0.293255   
75%     765.000000   765.000000   61.593253   61.357186      49.560117   
max    1020.000000  1020.000000  133.137830  132.742236      99.413490   

       turn_speed  
count  441.000000  
mean     0.191122  
std     35.025471  
min    -98.633769  
25%    -23.713031  
50%      0.146413  
75%     24.020476  
max     99.804497  

First few rows:
   x    y   lw_speed    rw_speed  forward_speed  turn_speed
0  0    0 -66.666667 -133.333333    -1

In [4]:
# Prepare data for 3D plotting
# Create pivot tables for each metric
x_vals = sorted(df['x'].unique())
y_vals = sorted(df['y'].unique())

# Create meshgrid
X, Y = np.meshgrid(x_vals, y_vals)

# Pivot data for each speed component
lw_speed_pivot = df.pivot(index='y', columns='x', values='lw_speed')
rw_speed_pivot = df.pivot(index='y', columns='x', values='rw_speed')
forward_speed_pivot = df.pivot(index='y', columns='x', values='forward_speed')
turn_speed_pivot = df.pivot(index='y', columns='x', values='turn_speed')

print(f"Grid dimensions: {len(x_vals)} x {len(y_vals)}")

Grid dimensions: 21 x 21


In [5]:
# Create 3D surface plots using Plotly
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=('Left Wheel Speed', 'Right Wheel Speed', 'Forward Speed', 'Turn Speed'),
    vertical_spacing=0.08
)

# Left wheel speed surface
fig.add_trace(
    go.Surface(z=lw_speed_pivot.values, x=x_vals, y=y_vals, colorscale='Viridis', name='Left Wheel'),
    row=1, col=1
)

# Right wheel speed surface
fig.add_trace(
    go.Surface(z=rw_speed_pivot.values, x=x_vals, y=y_vals, colorscale='Plasma', name='Right Wheel'),
    row=1, col=2
)

# Forward speed surface
fig.add_trace(
    go.Surface(z=forward_speed_pivot.values, x=x_vals, y=y_vals, colorscale='Blues', name='Forward'),
    row=2, col=1
)

# Turn speed surface
fig.add_trace(
    go.Surface(z=turn_speed_pivot.values, x=x_vals, y=y_vals, colorscale='RdBu', name='Turn'),
    row=2, col=2
)

fig.update_layout(
    title='SpeedController Behavior Analysis',
    height=800,
    showlegend=False
)

# Update axis labels
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_scenes(
            xaxis_title='Joystick X',
            yaxis_title='Joystick Y',
            zaxis_title='Speed',
            row=i, col=j
        )

fig.show()

In [6]:
# Create individual detailed plots
fig_individual = go.Figure()

# Left wheel speed
fig_lw = go.Figure(data=[go.Surface(
    z=lw_speed_pivot.values, 
    x=x_vals, 
    y=y_vals,
    colorscale='Viridis'
)])

fig_lw.update_layout(
    title='Left Wheel Speed vs Joystick Position',
    scene=dict(
        xaxis_title='Joystick X (0-1023)',
        yaxis_title='Joystick Y (0-1023)',
        zaxis_title='Left Wheel Speed (-100 to 100)'
    ),
    width=800,
    height=600
)

fig_lw.show()

In [7]:
# Right wheel speed
fig_rw = go.Figure(data=[go.Surface(
    z=rw_speed_pivot.values, 
    x=x_vals, 
    y=y_vals,
    colorscale='Plasma'
)])

fig_rw.update_layout(
    title='Right Wheel Speed vs Joystick Position',
    scene=dict(
        xaxis_title='Joystick X (0-1023)',
        yaxis_title='Joystick Y (0-1023)',
        zaxis_title='Right Wheel Speed (-100 to 100)'
    ),
    width=800,
    height=600
)

fig_rw.show()

In [8]:
# Turn speed analysis
fig_turn = go.Figure(data=[go.Surface(
    z=turn_speed_pivot.values, 
    x=x_vals, 
    y=y_vals,
    colorscale='RdBu'
)])

fig_turn.update_layout(
    title='Turn Speed vs Joystick Position',
    scene=dict(
        xaxis_title='Joystick X (0-1023)',
        yaxis_title='Joystick Y (0-1023)',
        zaxis_title='Turn Speed (-100 to 100)'
    ),
    width=800,
    height=600
)

fig_turn.show()

In [9]:
# Analysis: Show turn speed reduction at high forward speeds
# Filter data for center X position (no turn input)
center_x = 512  # Approximate center
center_data = df[df['x'] == center_x].copy()

# Filter data for maximum forward speed
max_y = df['y'].max()
max_forward_data = df[df['y'] == max_y].copy()

fig_analysis = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Turn Speed vs Forward Speed', 'Turn Speed at Max Forward')
)

# Plot 1: Turn speed vs forward speed relationship
fig_analysis.add_trace(
    go.Scatter(x=center_data['forward_speed'], y=center_data['turn_speed'], 
               mode='markers', name='No Turn Input'),
    row=1, col=1
)

# Plot 2: Turn speed across X positions at max forward
fig_analysis.add_trace(
    go.Scatter(x=max_forward_data['x'], y=max_forward_data['turn_speed'], 
               mode='lines+markers', name='Max Forward Speed'),
    row=1, col=2
)

fig_analysis.update_layout(
    title='Turn Speed Analysis',
    height=400
)

fig_analysis.show()

print(f"Turn speed reduction at max forward: {max_forward_data['turn_speed'].abs().max():.1f}")
print(f"Expected reduction to 1/3: {100/3:.1f}")

Turn speed reduction at max forward: 33.7
Expected reduction to 1/3: 33.3
